In [6]:
import os

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=""
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["OPENAI_API_VERSION"] = ""

project_name = "default"

# Log a trace via REST API

https://docs.smith.langchain.com/cookbook/tracing-examples/rest

In [65]:
import datetime
import uuid
import requests

_LANGSMITH_API_KEY = os.environ["LANGCHAIN_API_KEY"]

run_id = str(uuid.uuid4())

res = requests.post(
    "https://api.smith.langchain.com/runs",
    json={
        "id": run_id,
        "name": "MyFirstRun",
        "run_type": "chain",
        "start_time": datetime.datetime.utcnow().isoformat(),
        "inputs": {"text": "Foo"},
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)
print(res)


<Response [202]>


In [66]:
requests.patch(
    f"https://api.smith.langchain.com/runs/{run_id}",
    json={
        "outputs": {"my_output": "Bar"},
        "end_time": datetime.datetime.utcnow().isoformat(),
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)

<Response [202]>

More complex chain example

In [8]:
import platform
import datetime
import uuid
import requests

_LANGSMITH_API_KEY = os.environ["LANGCHAIN_API_KEY"]
run_id = str(uuid.uuid4())

requests.post(
    "https://api.smith.langchain.com/runs",
    json={
        "id": run_id,
        "name": "MySecondRun",
        "run_type": "chain",
        "inputs": {"text": "Foo"},
        "start_time": datetime.datetime.utcnow().isoformat(),
        "session_name": project_name,
        "tags": ["langsmith", "rest", "my-example"],
        "extra": {
            "metadata": {"my_key": "My value"},
            "runtime": {
                "platform": platform.platform(),
            },
        },
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)
# ... do some work ...
events = []
# Events like new tokens and retries can be added
events.append({"event_name": "retry", "reason": "never gonna give you up"})
events.append({"event_name": "new_token", "value": "foo"})

res = requests.patch(
    f"https://api.smith.langchain.com/runs/{run_id}",
    json={
        "end_time": datetime.datetime.utcnow().isoformat(),
        "outputs": {"generated": "Bar"},
        "events": events,
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)

In [9]:
import platform

run_id = str(uuid.uuid4())

requests.post(
    "https://api.smith.langchain.com/runs",
    json={
        "id": run_id,
        "name": "MySecondRun",
        "run_type": "chain",
        "inputs": {"text": "Foo"},
        "start_time": datetime.datetime.utcnow().isoformat(),
        "session_name": project_name,
        "tags": ["langsmith", "rest", "my-example"],
        "extra": {
            "metadata": {"my_key": "My value"},
            "runtime": {
                "platform": platform.platform(),
            },
        },
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)
# ... do some work ...
events = []
# Events like new tokens and retries can be added
events.append({"event_name": "retry", "reason": "never gonna give you up"})
events.append({"event_name": "new_token", "value": "foo"})

res = requests.patch(
    f"https://api.smith.langchain.com/runs/{run_id}",
    json={
        "end_time": datetime.datetime.utcnow().isoformat(),
        "outputs": {"generated": "Bar"},
        "events": events,
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)

In [10]:
run_id = str(uuid.uuid4())
requests.post(
    "https://api.smith.langchain.com/runs",
    json={
        "id": run_id,
        "name": "MyChatModelRun",
        "run_type": "llm",
        "inputs": {
            "messages": [{"role": "user", "content": "What's the weather in SF like?"}],
            # Optional
            "model": "text-davinci-003",
            "functions": [
                {
                    "name": "get_current_weather",
                    "description": "Get the current weather in a given location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "unit": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                            },
                        },
                        "required": ["location"],
                    },
                }
            ],
            # You can add other invocation paramers as k-v pairs
            "temperature": 0.0,
        },
        "start_time": datetime.datetime.utcnow().isoformat(),
        "session_name": project_name,
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)

requests.patch(
    f"https://api.smith.langchain.com/runs/{run_id}",
    json={
        "end_time": datetime.datetime.utcnow().isoformat(),
        "outputs": {
            "choices": [
                {
                    "index": 0,
                    "message": {
                        "role": "assistant",
                        # Content is whatever string response the
                        # model generates
                        "content": "Mostly cloudy.",
                        # Function call is the function invocation and arguments
                        # as a string
                        "function_call": {
                            "name": "get_current_weather",
                            "arguments": '{\n"location": "San Francisco, CA"\n}',
                        },
                    },
                    "finish_reason": "function_call",
                }
            ],
        },
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)

<Response [202]>

In [11]:
run_id = str(uuid.uuid4())
requests.post(
    "https://api.smith.langchain.com/runs",
    json={
        "id": run_id,
        "name": "MyLLMRun",
        "run_type": "llm",
        "inputs": {
            "prompt": "Hi there!",
            # Optional: model or engine name, and other invocation params
            "engine": "text-davinci-003",
            "temperature": 0.0,
        },
        "start_time": datetime.datetime.utcnow().isoformat(),
        "session_name": project_name,
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)

requests.patch(
    f"https://api.smith.langchain.com/runs/{run_id}",
    json={
        "end_time": datetime.datetime.utcnow().isoformat(),
        "outputs": {
            "choices": [
                {
                    "text": "\nMy name is Polly and I'm excited to talk to you!",
                    "index": 0,
                    "logprobs": None,
                    "finish_reason": "stop",
                },
            ]
        },
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)

<Response [202]>

Nesting Runs

In [12]:
import uuid
from typing import Optional


class RunLogger:
    def post_run(
        self, data: dict, name: str, run_id: str, parent_run_id: Optional[str] = None
    ) -> None:
        requests.post(
            "https://api.smith.langchain.com/runs",
            json={
                "id": run_id,
                "name": name,
                "run_type": "chain",
                "parent_run_id": parent_run_id,
                "inputs": data,
                "start_time": datetime.datetime.utcnow().isoformat(),
                "session_name": project_name,
            },
            headers={"x-api-key": _LANGSMITH_API_KEY},
        )

    def patch_run(
        self, run_id: str, output: Optional[dict] = None, error: Optional[str] = None
    ) -> None:
        requests.patch(
            f"https://api.smith.langchain.com/runs/{run_id}",
            json={
                "error": error,
                "outputs": output,
                "end_time": datetime.datetime.utcnow().isoformat(),
            },
            headers={"x-api-key": _LANGSMITH_API_KEY},
        )

In [13]:
logger = RunLogger()


def fibonacci(n: int, depth: int = 0, parent_run_id: Optional[str] = None) -> int:
    run_id = str(uuid.uuid4())
    logger.post_run(
        {"n": n}, f"fibonacci_recursive", run_id, parent_run_id=parent_run_id
    )
    try:
        if n <= 1:
            result = n
        else:
            result = fibonacci(n - 1, depth + 1, parent_run_id=run_id) + fibonacci(
                n - 2, depth + 1, parent_run_id=run_id
            )
        logger.patch_run(run_id, output={"result": result})
        return result
    except Exception as e:
        logger.patch_run(run_id, error=str(e))
        raise

In [14]:
fibonacci(5)

5

In [15]:
def fibonacci(n: int, depth: int = 0, parent_run_id: Optional[str] = None) -> int:
    run_id = str(uuid.uuid4())
    logger.post_run(
        {"n": n}, f"fibonacci_recursive", run_id, parent_run_id=parent_run_id
    )
    try:
        if n < 0:
            raise ValueError("NEGATIVE NUMER NOT ALLOWED")
        if n <= 1:
            result = n
        else:
            result = fibonacci(n - 1, depth + 1, parent_run_id=run_id) + fibonacci(
                n - 2, depth + 1, parent_run_id=run_id
            )
        logger.patch_run(run_id, output={"result": result})
        return result
    except Exception as e:
        logger.patch_run(run_id, error=str(e))
        raise


# We will show what the trace looks like with an error
try:
    fibonacci(2.3)
except:
    pass

Single Requests

In [16]:
requests.post(
    "https://api.smith.langchain.com/runs",
    json={
        "name": "MyLLMRun",
        "run_type": "llm",
        "inputs": {
            "prompt": "Hi there!",
            # Optional: model or engine name, and other invocation params
            "engine": "text-davinci-003",
            "temperature": 0.0,
        },
        "outputs": {
            "choices": [
                {
                    "text": "\nMy name is Polly and I'm excited to talk to you!",
                    "index": 0,
                    "logprobs": None,
                    "finish_reason": "stop",
                },
            ]
        },
        "session_name": project_name,
        "start_time": datetime.datetime.utcnow().isoformat(),
        "end_time": datetime.datetime.utcnow().isoformat(),
    },
    headers={"x-api-key": _LANGSMITH_API_KEY},
)

<Response [202]>